In [30]:
import numpy as np
import pymc3 as pm
import theano.tensor as tt

In [31]:
# Create some fake data
ntrials_small = 200
ntrials_large = 1000
ntimes = 3
ninputs = 6

y_small = np.random.normal(size=(ntrials_small,ntimes))
x_small = np.random.normal(size=(ntrials_small,ninputs))
y_small_flat = np.ndarray.flatten(y_small) 

y_large = np.random.normal(size=(ntrials_large,ntimes))
x_large = np.random.normal(size=(ntrials_large,ninputs))
y_large_flat = np.ndarray.flatten(y_large) 

In [32]:
# Define pymc models

# MatrixNormal, Small data
with pm.Model() as mn_small:
    
    # Priors
    muvec = pm.Normal('muvec', shape=ntimes)
    sigma = pm.HalfCauchy('sigma', beta=10., testval=0.5)
    packed_L = pm.LKJCholeskyCov('packed_L', n=ntimes, eta=2., sd_dist=pm.HalfCauchy.dist(2.5))
    ell = pm.HalfCauchy('ell', beta=10., testval=0.5, shape=ninputs)

    # Covariance function/matrices
    cor = pm.gp.cov.ExpQuad(input_dim=ninputs, ls=ell)
    K = cor(x_small)
    L = pm.expand_packed_triangular(ntimes, packed_L)

    # Mean matrix
    M = pm.Deterministic('M', tt.tile(muvec,(y_small.shape[0],1)))

    # Likelihood
    y = pm.MatrixNormal('y', mu=M, rowcov=K, colchol=L, observed=y_small, shape=y_small.shape)


# KroneckerNormal, Small data
with pm.Model() as kn_small:
    
    # Priors
    muvec = pm.Normal('muvec', shape=ntimes)
    sigma = pm.HalfCauchy('sigma', beta=10., testval=0.5)
    packed_L = pm.LKJCholeskyCov('packed_L', n=ntimes, eta=2., sd_dist=pm.HalfCauchy.dist(2.5))
    ell = pm.HalfCauchy('ell', beta=10., testval=0.5, shape=ninputs)

    # Covariance function/matrices
    cor = pm.gp.cov.ExpQuad(input_dim=ninputs, ls=ell)
    K = cor(x_small)
    L = pm.expand_packed_triangular(ntimes, packed_L)
    S = L.dot(L.T)
    evds = [tt.nlinalg.eigh(Ki) for Ki in [K, S]]

    # Mean function
    M = pm.Deterministic('M', tt.repeat(muvec,y_small.shape[0]))

    # Likelihood
    y = pm.KroneckerNormal('y', mu=M, evds=evds, observed=y_small_flat, sigma=sigma)
  
    
# MatrixNormal, Large data
with pm.Model() as mn_large:
    
    # Priors
    muvec = pm.Normal('muvec', shape=ntimes)
    sigma = pm.HalfCauchy('sigma', beta=10., testval=0.5)
    packed_L = pm.LKJCholeskyCov('packed_L', n=ntimes, eta=2., sd_dist=pm.HalfCauchy.dist(2.5))
    ell = pm.HalfCauchy('ell', beta=10., testval=0.5, shape=ninputs)

    # Covariance function/matrices
    cor = pm.gp.cov.ExpQuad(input_dim=ninputs, ls=ell)
    K = cor(x_large)
    L = pm.expand_packed_triangular(ntimes, packed_L)

    # Mean matrix
    M = pm.Deterministic('M', tt.tile(muvec,(y_large.shape[0],1)))

    # Likelihood
    y = pm.MatrixNormal('y', mu=M, rowcov=K, colchol=L, observed=y_large, shape=y_large.shape)
    

# KroneckerNormal, Large data
with pm.Model() as kn_large:
    
    # Priors
    muvec = pm.Normal('muvec', shape=ntimes)
    sigma = pm.HalfCauchy('sigma', beta=10., testval=0.5)
    packed_L = pm.LKJCholeskyCov('packed_L', n=ntimes, eta=2., sd_dist=pm.HalfCauchy.dist(2.5))
    ell = pm.HalfCauchy('ell', beta=10., testval=0.5, shape=ninputs)

    # Covariance function/matrices
    cor = pm.gp.cov.ExpQuad(input_dim=ninputs, ls=ell)
    K = cor(x_large)
    L = pm.expand_packed_triangular(ntimes, packed_L)
    S = L.dot(L.T)
    evds = [tt.nlinalg.eigh(Ki) for Ki in [K, S]]

    # Mean function
    M = pm.Deterministic('M', tt.repeat(muvec,y_large.shape[0]))

    # Likelihood
    y = pm.KroneckerNormal('y', mu=M, evds=evds, observed=y_large_flat, sigma=sigma)

In [34]:
# Profile small models
mn_small.profile(mn_small.logpt).summary()

Function profiling
  Message: /Users/neklein/Python/Environments/rdxgp/lib/python3.6/site-packages/pymc3/model.py:903
  Time in 1000 calls to Function.__call__: 1.465944e+00s
  Time in Function.fn.__call__: 1.413333e+00s (96.411%)
  Time in thunks: 1.366755e+00s (93.234%)
  Total compile time: 4.073930e-01s
    Number of Apply nodes: 70
    Theano Optimizer time: 2.757020e-01s
       Theano validate time: 8.796930e-03s
    Theano Linker time (includes C, CUDA code generation/compiling): 6.367517e-02s
       Import time 0.000000e+00s
       Node make_thunk time 6.091809e-02s
           Node Elemwise{Sub}[(0, 1)](TensorConstant{[[ 5.42204..0886e-01]]}, M) time 2.803087e-02s
           Node Solve{A_structure='lower_triangular', lower=True, overwrite_A=False, overwrite_b=False}(AdvancedIncSubtensor{inplace=False,  set_instead_of_inc=True}.0, Dot22.0) time 1.236916e-03s
           Node InplaceDimShuffle{x,0}(Sum{axis=[1], acc_dtype=float64}.0) time 9.217262e-04s
           Node InplaceDimSh

In [35]:
# Profile small models
kn_small.profile(kn_small.logpt).summary()

Function profiling
  Message: /Users/neklein/Python/Environments/rdxgp/lib/python3.6/site-packages/pymc3/model.py:903
  Time in 1000 calls to Function.__call__: 9.589539e+00s
  Time in Function.fn.__call__: 9.534008e+00s (99.421%)
  Time in thunks: 9.478535e+00s (98.842%)
  Total compile time: 6.197667e-01s
    Number of Apply nodes: 77
    Theano Optimizer time: 3.953729e-01s
       Theano validate time: 1.238537e-02s
    Theano Linker time (includes C, CUDA code generation/compiling): 1.625328e-01s
       Import time 0.000000e+00s
       Node make_thunk time 1.592040e-01s
           Node for{cpu,scan_fn}(TensorConstant{1}, Elemwise{Sub}[(0, 1)].0, TensorConstant{1}, ell, packed_L, Alloc.0) time 9.073091e-02s
           Node Elemwise{Sub}[(0, 1)](TensorConstant{[[ 5.42204..0886e-01]]}, InplaceDimShuffle{x,0}.0) time 2.813411e-02s
           Node MakeVector{dtype='float64'}(__logp_muvec, __logp_sigma_log__, __logp_packed_L_cholesky_cov_packed__, __logp_ell_log__, __logp_y) time 1.67512

In [36]:
# Profile large models
mn_large.profile(mn_large.logpt).summary()

Function profiling
  Message: /Users/neklein/Python/Environments/rdxgp/lib/python3.6/site-packages/pymc3/model.py:903
  Time in 1000 calls to Function.__call__: 2.593426e+01s
  Time in Function.fn.__call__: 2.587368e+01s (99.766%)
  Time in thunks: 2.475416e+01s (95.450%)
  Total compile time: 4.145560e-01s
    Number of Apply nodes: 70
    Theano Optimizer time: 2.766502e-01s
       Theano validate time: 9.714842e-03s
    Theano Linker time (includes C, CUDA code generation/compiling): 7.350516e-02s
       Import time 0.000000e+00s
       Node make_thunk time 7.057309e-02s
           Node Elemwise{Sub}[(0, 1)](TensorConstant{[[-0.29761..21279899]]}, M) time 2.876592e-02s
           Node InplaceDimShuffle{x,0}(ell) time 1.753092e-03s
           Node InplaceDimShuffle{x}(Shape_i{0}.0) time 1.728058e-03s
           Node Cholesky{lower=True, destructive=False, nofail=False}(Elemwise{Composite{exp((i0 * clip((i1 + i2 + i3), i4, i5)))}}[(0, 1)].0) time 1.715899e-03s
           Node Elemwise

In [37]:
# Profile large models
kn_large.profile(kn_large.logpt).summary()

Function profiling
  Message: /Users/neklein/Python/Environments/rdxgp/lib/python3.6/site-packages/pymc3/model.py:903
  Time in 1000 calls to Function.__call__: 3.076107e+02s
  Time in Function.fn.__call__: 3.075431e+02s (99.978%)
  Time in thunks: 3.064911e+02s (99.636%)
  Total compile time: 6.724150e-01s
    Number of Apply nodes: 77
    Theano Optimizer time: 4.298701e-01s
       Theano validate time: 1.291370e-02s
    Theano Linker time (includes C, CUDA code generation/compiling): 1.755798e-01s
       Import time 0.000000e+00s
       Node make_thunk time 1.720772e-01s
           Node for{cpu,scan_fn}(TensorConstant{1}, Elemwise{Sub}[(0, 1)].0, TensorConstant{1}, ell, packed_L, Alloc.0) time 1.002209e-01s
           Node Elemwise{Sub}[(0, 1)](TensorConstant{[[-0.29761..21279899]]}, InplaceDimShuffle{x,0}.0) time 2.564907e-02s
           Node Elemwise{Mul}[(0, 1)](TensorConstant{0.5}, Sum{acc_dtype=float64}.0) time 2.330303e-03s
           Node Elemwise{Sqr}[(0, 0)](InplaceDimShuff